# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a short definition of its context we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
# Function to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=context,
            temperature=0.5,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['customer', 'purchase', 'lines'],
    'definition': [
        'Represents a single customer entity (id, first_name, last_name, email, created_at).',
        'Represents a purchase made by a customer (purchase_id, customer_id, purchase_date, amount, currency, additional_costs, total).',
        'Represents line items within a purchase (line_id, purchase_id, product_id, product_name, quantity, line_amount, taxes, voucher_applied, points_applied).'
    ]
}
df = pd.DataFrame(data)
print(df)

      table                                         definition
0  customer  Represents a single customer entity (id, first...
1  purchase  Represents a purchase made by a customer (purc...
2     lines  Represents line items within a purchase (line_...


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [5]:
print(text_tables)

customer: Represents a single customer entity (id, first_name, last_name, email, created_at).
purchase: Represents a purchase made by a customer (purchase_id, customer_id, purchase_date, amount, currency, additional_costs, total).
lines: Represents line items within a purchase (line_id, purchase_id, product_id, product_name, quantity, line_amount, taxes, voucher_applied, points_applied).


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""


In [11]:
# Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='I want to know the total amount of products sold per customer')

In [12]:
print(return_OAI(pqt1))

{
  "tables": ["customer", "purchase", "lines"]
}


In [13]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question='I want to know the total amount of a single product sold across all customers')

In [14]:
print(return_OAI(pqt3))

{
  "tables": ["lines"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?